airdoorのスクレイピング

In [52]:
import requests
from bs4 import BeautifulSoup
import time
import re

REQUEST_URL = 'https://airdoor.jp/list?si=d-131091&p={}'

res = requests.get(REQUEST_URL)
max_pages = 10

# 複数ページの情報をまとめて取得
data_samples = []

for page in range(1, max_pages + 1):
    # ページ情報
    url = REQUEST_URL.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  'lxml')
    time.sleep(1)

    # 物件情報リストを指定
    mother = soup.find_all(class_='PropertyPanel_propertyPanel__MqCpF')

    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []

        def extract_room_info(room_info):
        # 部屋番号、間取り、広さ、方角を正規表現で抽出
            room_matches = re.match(r'(\d+)号室 \/ (\S+) \/ (\d+(\.\d+)?)㎡ \/ (.+)', room_info)

            if room_matches:
            # room_numberのクレンジング処理
                room_number_raw = room_matches.group(1)
                room_number = str(int(room_number_raw))  # 半角数字に変換
                return room_number, room_matches.group(2), room_matches.group(3), room_matches.group(5)
            else:
                return "", "", "", ""

        def extract_address_info(building_info):
        # 住所情報を正規表現で抽出
            address_matches = re.findall(r'(東京都|大阪府|京都府|などの都道府県の正規表現)(.*?)(\d+丁目|$)', building_info)

            if address_matches:
                prefecture = address_matches[0][0]
                city = address_matches[0][1].replace('の', '')  # 区市町村から不要な文字を削除
                address = address_matches[0][2]
                return prefecture, city, address
            else:
                return "", "", ""

        # 建物名
        building_info = child.find(class_='PropertyPanelBuilding_buildingTitle__NbWmb').text
        building_info_parts = re.split(r'\s+', building_info)  # 空白で分割
        vacancy_info = building_info_parts[0] if building_info_parts else ""
        building_name = building_info_parts[1] if len(building_info_parts) > 1 else ""

        data_home.append(vacancy_info)
        data_home.append(building_name)

        # 家賃、管理費
        rent_info = child.find(class_='PropertyPanelRoom_rentPrice__HO4Jp').text
        rent_matches = re.match(r'(\d+,\d+)円 \((.+)\)', rent_info)
        rent = rent_matches.group(1) if rent_matches else ""
        management_fee = rent_matches.group(2) if rent_matches else ""

        # 1. rent の末尾に「円」を付ける
        rent += "円"

        # 2. management_fee から文頭の「管理費」を取り除く
        management_fee = management_fee.replace("管理費", "").replace("円", "")

        # 3. rent と management_fee を足して total_fee を項目として追加する
        total_fee = str(int(rent.replace("円", "").replace(",", "")) + int(management_fee.replace(",", "")))
        total_fee = f'{int(total_fee):,}円'

        # 4. 管理費にも円を付け足す
        management_fee += "円"

        data_home.append(rent)
        data_home.append(management_fee)
        data_home.append(total_fee)

        # 部屋情報（間取り）
        room_info = child.find(class_='is-ml5').text

        # 部屋情報を正規表現で抽出
        room_number, layout, area, direction = extract_room_info(room_info)

        # 最寄り駅のアクセス、住所、建物情報、築年数、階数
        building_info = child.find(class_='PropertyPanelBuilding_buildingInformationSection__AMRsh').text

        # 住所情報を正規表現で抽出
        prefecture, city, address = extract_address_info(building_info)

        # 正規表現を使って情報を抽出
        walk_time_matches = re.findall(r'徒歩(\d+)分', building_info)
        delivery_box_matches = re.findall(r'宅配ボックス', building_info)
        parking_matches = re.findall(r'駐車場あり', building_info)
        elevator_matches = re.findall(r'エレベーター', building_info)

        # 住所、徒歩、宅配ボックス、駐車場あり、エレベーターの情報を取得
        walk_time = walk_time_matches[0] if walk_time_matches else ""
        delivery_box = "〇" if delivery_box_matches else ""
        parking = "〇" if parking_matches else ""
        elevator = "〇" if elevator_matches else ""

        # 最寄り駅のアクセス、住所、建物情報、築年数、階数
        building_info2 = child.find(class_='PropertyPanelBuilding_buildingInformation__kRI1U is-pc-only').text

        # 築年数、階数、材質情報を正規表現で抽出
        built_year_matches = re.search(r'築(\d+年|新築)', building_info2)
        floors_matches = re.search(r'(\d+)階建', building_info2)
        material_matches = re.search(r'(鉄筋コンクリート|鉄骨造|木造|軽量鉄骨|その他)', building_info2)

        # 築年数、階数、材質情報を変数に格納
        built_year = built_year_matches.group(1) if built_year_matches else "新築"
        floors = floors_matches.group(1) if floors_matches else ""
        material = material_matches.group(1) if material_matches else ""

        # お得情報(項目)
        sale_info_element = child.find(class_='PropertyPanelRoom_isRed__HXtHO PropertyPanelRoom_isBold__hkYt7')
        sale_info = sale_info_element.text if sale_info_element else ""

        # お得情報(額面)
        sale_info_element_2 = child.find(class_='PropertyPanelRoom_strikeText__N_r0P')
        sale_info_2 = sale_info_element_2.text if sale_info_element_2 else ""

        # お得情報(概要)を取得
        sale_info_element_3 = child.find(class_='PropertyPanelRoom_freerentTagBlock__n_YOv')
        sale_info_3 = sale_info_element_3.text if sale_info_element_3 else ""

        # お得情報(星)を取得
        stars_container = child.find(class_='InitialCostScore_starsContainer__q41Lt')

        # おすすめ度の星の数を数える
        recommended_stars = stars_container.find_all('span', class_='Stars_starSvg__NcvTr Stars_active__7B_Zp') if stars_container else []
        not_recommended_stars = stars_container.find_all('span', class_='Stars_starSvg__NcvTr Stars_inActive__u6vwG') if stars_container else []

        # おすすめ度の数をカウント
        recommended_count = len(recommended_stars)
        not_recommended_count = len(not_recommended_stars)

        # カラムに情報を追加
        data_home.extend([room_number, layout, area, direction, prefecture, city, address, walk_time, delivery_box, parking, elevator, built_year, floors, material, sale_info , sale_info_2, sale_info_3, recommended_count])

        # 物件情報と部屋情報をくっつける
        data_samples.append(data_home)

In [53]:
import pandas as pd
df = pd.DataFrame(data_samples)
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,【空室95件】,Ｊ．ＧＲＡＮ,"113,000円","10,000円","123,000円",308,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"124,300円",フリーレント2ヶ月(22.6万円お得),3
1,【空室1件】,プロト不動前,"77,000円","13,000円","90,000円",201,1R,15.61,北東,東京都,...,〇,,,5年,4,鉄筋コンクリート,仲介手数料無料,"84,700円",,2
2,【空室4件】,アジールコート品川中延サウス,"109,000円","10,000円","119,000円",404,1K,25.4,北,東京都,...,〇,,,新築,8,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント1ヶ月(11.2万円お得),2
3,【空室3件】,プラチナフォルム中延EAST,"83,000円","10,000円","93,000円",302,1R,18.28,北,東京都,...,〇,,,2年,4,鉄筋コンクリート,仲介手数料無料,"91,300円",フリーレント1ヶ月(8.3万円お得),3
4,【空室2件】,プラウドフラット大森Ⅲ,"115,000円","8,000円","123,000円",408,1K,25.89,南西,東京都,...,〇,,〇,9年,8,鉄筋コンクリート,仲介手数料無料,"126,500円",フリーレント1ヶ月(11.5万円お得),3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,【空室1件】,プレール・ドゥーク品川御殿山,"81,000円","10,000円","91,000円",303,1K,22.52,南,東京都,...,〇,,〇,20年,4,鉄筋コンクリート,"44,000円","89,100円",,0
196,【空室2件】,Ｌ-Ｆｌａｔ南品川,"82,000円","6,000円","88,000円",207,1K,22.5,北,東京都,...,〇,,〇,17年,7,鉄筋コンクリート,"43,000円","90,200円",,0
197,【空室1件】,ヴェルト五反田西,"80,000円","9,000円","89,000円",202,1K,22.31,南東,東京都,...,〇,,〇,20年,13,,"44,000円","88,000円",,0
198,【空室1件】,ロイヤル高輪台,"170,000円",0円,"170,000円",304,1SDK,50.24,南西,東京都,...,〇,,〇,34年,3,鉄筋コンクリート,"88,000円","187,000円",,1


In [54]:
df = df.rename(columns = {0:'空室情報',1:'建物名',2:'家賃',3:'管理費',4:'合計金額',5:'部屋No',6:'間取り',7:'面積',8:'方角',9:'都道府県',10:'市区町村',11:'丁目',12:'駅徒歩',13:'宅配ボックス',14:'駐車場',15:'エレベーター',16:'築年情報',17:'物件階数',18:'材質',19:'お得情報',20:'値引金額',21:'追加セールスポイント',22:'おすすめ度'})
df

,空室情報,建物名,家賃,管理費,合計金額,部屋No,間取り,面積,方角,都道府県,...,宅配ボックス,駐車場,エレベーター,築年情報,物件階数,材質,お得情報,値引金額,追加セールスポイント,おすすめ度
0,【空室95件】,Ｊ．ＧＲＡＮ,"113,000円","10,000円","123,000円",308,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"124,300円",フリーレント2ヶ月(22.6万円お得),3
1,【空室1件】,プロト不動前,"77,000円","13,000円","90,000円",201,1R,15.61,北東,東京都,...,〇,,,5年,4,鉄筋コンクリート,仲介手数料無料,"84,700円",,2
2,【空室4件】,アジールコート品川中延サウス,"109,000円","10,000円","119,000円",404,1K,25.4,北,東京都,...,〇,,,新築,8,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント1ヶ月(11.2万円お得),2
3,【空室3件】,プラチナフォルム中延EAST,"83,000円","10,000円","93,000円",302,1R,18.28,北,東京都,...,〇,,,2年,4,鉄筋コンクリート,仲介手数料無料,"91,300円",フリーレント1ヶ月(8.3万円お得),3
4,【空室2件】,プラウドフラット大森Ⅲ,"115,000円","8,000円","123,000円",408,1K,25.89,南西,東京都,...,〇,,〇,9年,8,鉄筋コンクリート,仲介手数料無料,"126,500円",フリーレント1ヶ月(11.5万円お得),3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,【空室1件】,プレール・ドゥーク品川御殿山,"81,000円","10,000円","91,000円",303,1K,22.52,南,東京都,...,〇,,〇,20年,4,鉄筋コンクリート,"44,000円","89,100円",,0
196,【空室2件】,Ｌ-Ｆｌａｔ南品川,"82,000円","6,000円","88,000円",207,1K,22.5,北,東京都,...,〇,,〇,17年,7,鉄筋コンクリート,"43,000円","90,200円",,0
197,【空室1件】,ヴェルト五反田西,"80,000円","9,000円","89,000円",202,1K,22.31,南東,東京都,...,〇,,〇,20年,13,,"44,000円","88,000円",,0
198,【空室1件】,ロイヤル高輪台,"170,000円",0円,"170,000円",304,1SDK,50.24,南西,東京都,...,〇,,〇,34年,3,鉄筋コンクリート,"88,000円","187,000円",,1


In [55]:
# '面積'が欠損している行を削除
df = df.dropna(subset=['面積'])

# 重複が削除できていないようであれば、再度実行
df = df.drop_duplicates(subset=['建物名', '合計金額', '面積', '部屋No'])
df.drop_duplicates(inplace=True)
df

,空室情報,建物名,家賃,管理費,合計金額,部屋No,間取り,面積,方角,都道府県,...,宅配ボックス,駐車場,エレベーター,築年情報,物件階数,材質,お得情報,値引金額,追加セールスポイント,おすすめ度
0,【空室95件】,Ｊ．ＧＲＡＮ,"113,000円","10,000円","123,000円",308,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"124,300円",フリーレント2ヶ月(22.6万円お得),3
1,【空室1件】,プロト不動前,"77,000円","13,000円","90,000円",201,1R,15.61,北東,東京都,...,〇,,,5年,4,鉄筋コンクリート,仲介手数料無料,"84,700円",,2
2,【空室4件】,アジールコート品川中延サウス,"109,000円","10,000円","119,000円",404,1K,25.4,北,東京都,...,〇,,,新築,8,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント1ヶ月(11.2万円お得),2
3,【空室3件】,プラチナフォルム中延EAST,"83,000円","10,000円","93,000円",302,1R,18.28,北,東京都,...,〇,,,2年,4,鉄筋コンクリート,仲介手数料無料,"91,300円",フリーレント1ヶ月(8.3万円お得),3
4,【空室2件】,プラウドフラット大森Ⅲ,"115,000円","8,000円","123,000円",408,1K,25.89,南西,東京都,...,〇,,〇,9年,8,鉄筋コンクリート,仲介手数料無料,"126,500円",フリーレント1ヶ月(11.5万円お得),3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,【空室1件】,プレール・ドゥーク品川御殿山,"81,000円","10,000円","91,000円",303,1K,22.52,南,東京都,...,〇,,〇,20年,4,鉄筋コンクリート,"44,000円","89,100円",,0
196,【空室2件】,Ｌ-Ｆｌａｔ南品川,"82,000円","6,000円","88,000円",207,1K,22.5,北,東京都,...,〇,,〇,17年,7,鉄筋コンクリート,"43,000円","90,200円",,0
197,【空室1件】,ヴェルト五反田西,"80,000円","9,000円","89,000円",202,1K,22.31,南東,東京都,...,〇,,〇,20年,13,,"44,000円","88,000円",,0
198,【空室1件】,ロイヤル高輪台,"170,000円",0円,"170,000円",304,1SDK,50.24,南西,東京都,...,〇,,〇,34年,3,鉄筋コンクリート,"88,000円","187,000円",,1


In [56]:
#エクセルに落として重複がないか確認（確認をした後に非表示化）
df.to_excel('output_file2.xlsx', index=False, engine='openpyxl')

Google スプレッドシートに情報を転送する

In [28]:
#必要なライブラリをインポート
import gspread
from google.auth import exceptions
from google.oauth2.service_account import Credentials

In [29]:
# サービスアカウント キーを使用して認証情報を作成
credentials = Credentials.from_service_account_file('smooth-ocean-406910-dc84e296dda8.json', scopes=['https://www.googleapis.com/auth/spreadsheets'])
gc = gspread.authorize(credentials)

In [30]:
# スプレッドシートのURLからワークシートを取得
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1JT6fZwPAl_LWajo7iqqCjH4MKsYYW5dXgNxQQxId7ew/edit'
worksheet = gc.open_by_url(spreadsheet_url).sheet1

In [31]:
# データフレームをGoogleスプレッドシートに書き込む
worksheet.clear()  # シートをクリア
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

C:\Users\masay\AppData\Local\Temp\ipykernel_7688\4288850697.py:3: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1JT6fZwPAl_LWajo7iqqCjH4MKsYYW5dXgNxQQxId7ew',
 'updatedRange': "'シート1'!A1:W201",
 'updatedRows': 201,
 'updatedColumns': 23,
 'updatedCells': 4623}